In [12]:

import sys 
sys.path.append("/Users/Jessica/Desktop/Coding/School/2802ICT/Assignment2")
from imports import *

In [13]:
d = SetUp("car.csv", {'buying':'string','maint':'string','doors':'string','persons':'string','lug_boot':'string','safety':'string','class':'string'}) 
print(d.df.tail())


     buying maint  doors persons lug_boot safety  class
1723    low   low  5more    more      med    med   good
1724    low   low  5more    more      med   high  vgood
1725    low   low  5more    more      big    low  unacc
1726    low   low  5more    more      big    med   good
1727    low   low  5more    more      big   high  vgood


AttributeError: module 'numpy' has no attribute 'read'